# Saving to db from EODHD

In [6]:
import os
import sqlite3
import pandas as pd
import requests

# EOD API Token and URL
YOUR_API_TOKEN = '64ba54ca4a54b5.70340846'
eod_url = f'https://eodhd.com/api/eod-bulk-last-day/US?api_token={YOUR_API_TOKEN}&fmt=json'

# Define the SQLite database file path
db_path = "..\\data\\stock_data.db"  # Replace with your desired directory and file name
os.makedirs(os.path.dirname(db_path), exist_ok=True)  # Ensure the directory exists

# Function to save DataFrame to SQLite database
def save_to_db(df, connection):
    try:
        # Rename columns for SQLite compatibility
        df.rename(columns={
            "adjusted_close": "Adj_Close",
            "date": "Date",
            "open": "Open",
            "high": "High",
            "low": "Low",
            "close": "Close",
            "volume": "Volume",
            "code": "Ticker"
        }, inplace=True)

        # Ensure DataFrame has all required columns
        df = df[["Date", "Ticker", "Open", "High", "Low", "Close", "Adj_Close", "Volume"]]

        # Save to SQLite database
        df.to_sql("stock_data", connection, if_exists="append", index=False)
        print("Data successfully saved to the database.")

    except Exception as e:
        print(f"Error saving data to database: {str(e)}")

# Function to download data from EOD API and save to SQLite
def download_eod_data(connection):
    try:
        print("Downloading data from EOD API...")

        # Fetch data from the API
        response = requests.get(eod_url)
        response.raise_for_status()
        data = response.json()

        # Convert to DataFrame
        df = pd.DataFrame(data)

        # Save to database
        save_to_db(df, connection)

    except Exception as e:
        print(f"Error downloading EOD data: {str(e)}")

# Main function
def main():
    print("Starting stock data download from EOD API...")

    # Connect to SQLite database
    with sqlite3.connect(db_path) as conn:
        # Create table schema if it doesn't exist
        conn.execute("""
        CREATE TABLE IF NOT EXISTS stock_data (
            Date TEXT,
            Ticker TEXT,
            Open REAL,
            High REAL,
            Low REAL,
            Close REAL,
            Adj_Close REAL,
            Volume INTEGER
        );
        """)

        # Download EOD API data
        download_eod_data(conn)

    print("\nDownload Summary:")
    print("All stock data has been downloaded and saved successfully.")

if __name__ == "__main__":
    main()

Starting stock data download from EOD API...
Data successfully saved to the database.

Download Summary:
All stock data has been downloaded and saved successfully.


In [10]:
import sqlite3
import pandas as pd

# connect to the database
# con = sqlite3.connect(r"Z:\Data\sp500_data\Scripts\sp500_market_data.db")
con = sqlite3.connect(r"C:\Users\Jonat\Documents\GitHub\lstm-stock-price-prediction\data\stock_data.db")

# pd.read_sql_query("SELECT * FROM stock_data WHERE symbol = 'PANW'", con)
df = pd.read_sql_query("SELECT * FROM stock_data WHERE Ticker LIKE 'GOOGL'", con)
df = pd.read_sql_query("SELECT * FROM stock_data ", con)
con.close()
df

,Date,Ticker,Open,High,Low,Close,Adj_Close,Volume
0,2010-01-04 00:00:00,GOOGL,15.689439,15.753504,15.621622,15.684434,15.627782,78169752
1,2010-01-05 00:00:00,GOOGL,15.695195,15.711712,15.554054,15.615365,15.558963,120067812
2,2010-01-06 00:00:00,GOOGL,15.662162,15.662162,15.174174,15.221722,15.166740,158988852
3,2010-01-07 00:00:00,GOOGL,15.250250,15.265265,14.831081,14.867367,14.813666,256315428
4,2010-01-08 00:00:00,GOOGL,14.814815,15.096346,14.742492,15.065566,15.011149,188783028
...,...,...,...,...,...,...,...,...
3760,2024-12-11 00:00:00,GOOGL,185.309998,195.610001,184.850006,195.399994,195.399994,67894100
3761,2024-12-12 00:00:00,GOOGL,195.000000,195.179993,191.710007,191.960007,191.960007,34817500
3762,2024-12-13 00:00:00,GOOGL,191.009995,192.729996,189.639999,189.820007,189.820007,25143500
3763,2024-12-16 00:00:00,GOOGL,192.869995,199.000000,192.619995,196.660004,196.660004,44934900


In [8]:
df[df['Ticker'] == 'GOOGL']

,Date,Ticker,Open,High,Low,Close,Adj_Close,Volume
0,2010-01-04 00:00:00,GOOGL,15.689439,15.753504,15.621622,15.684434,15.627782,78169752
1,2010-01-05 00:00:00,GOOGL,15.695195,15.711712,15.554054,15.615365,15.558963,120067812
2,2010-01-06 00:00:00,GOOGL,15.662162,15.662162,15.174174,15.221722,15.166740,158988852
3,2010-01-07 00:00:00,GOOGL,15.250250,15.265265,14.831081,14.867367,14.813666,256315428
4,2010-01-08 00:00:00,GOOGL,14.814815,15.096346,14.742492,15.065566,15.011149,188783028
...,...,...,...,...,...,...,...,...
3760,2024-12-11 00:00:00,GOOGL,185.309998,195.610001,184.850006,195.399994,195.399994,67894100
3761,2024-12-12 00:00:00,GOOGL,195.000000,195.179993,191.710007,191.960007,191.960007,34817500
3762,2024-12-13 00:00:00,GOOGL,191.009995,192.729996,189.639999,189.820007,189.820007,25143500
3763,2024-12-16 00:00:00,GOOGL,192.869995,199.000000,192.619995,196.660004,196.660004,44934900


# Saving into sqlite db

In [1]:
import os
import sqlite3
import pandas as pd
import yfinance as yf
from tqdm import tqdm

# List of stock tickers to download
stock_list = ["GOOGL", "AAPL"]  # Add your desired tickers here

# Define the SQLite database file path
db_path = "..\\data\\stock_data.db"  # Replace with your desired directory and file name
os.makedirs(os.path.dirname(db_path), exist_ok=True)  # Ensure the directory exists

# Define the date range for stock data
start_date = '2010-01-01'
end_date = '2024-12-17'

# Function to download stock data and save it to SQLite
def download_and_save_to_db(ticker, connection):
    try:
        print(f"Downloading data for {ticker}...")

        # Download stock data
        stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        stock_data.columns = stock_data.columns.droplevel(1)

        # Validate that data was retrieved
        if stock_data.empty:
            raise ValueError(f"No data found for {ticker}")

        # Add a ticker column
        stock_data["Ticker"] = ticker

        # Reset index to make 'Date' a column
        stock_data.reset_index(inplace=True)

        # Rename columns for SQLite compatibility
        stock_data.rename(columns={
            "Adj Close": "Adj_Close",
            "Date": "Date",
            "Open": "Open",
            "High": "High",
            "Low": "Low",
            "Close": "Close",
            "Volume": "Volume"
        }, inplace=True)

        # Ensure correct column order and headers
        stock_data = stock_data[
            ["Date", "Ticker", "Open", "High", "Low", "Close", "Adj_Close", "Volume"]
        ]
        # print(stock_data)
        
        # Save to SQLite database
        stock_data.to_sql("stock_data", connection, if_exists="append", index=False)

        print(f"Data for {ticker} saved to database.")

    except Exception as e:
        print(f"Error downloading {ticker}: {str(e)}")

# Main function to iterate through tickers
def main():
    print("Starting sequential stock data download...")

    # Connect to SQLite database
    with sqlite3.connect(db_path) as conn:
        # Create table schema if it doesn't exist
        conn.execute("""
        CREATE TABLE IF NOT EXISTS stock_data (
            Date TEXT,
            Ticker TEXT,
            Open REAL,
            High REAL,
            Low REAL,
            Close REAL,
            Adj_Close REAL,
            Volume INTEGER
        );
        """)

        # Progress bar for the loop
        for ticker in tqdm(stock_list, desc="Downloading Stock Data"):
            download_and_save_to_db(ticker, conn)

    print("\nDownload Summary:")
    print("All stock data has been downloaded and saved successfully.")

if __name__ == "__main__":
    main()

Starting sequential stock data download...


Price       Date Ticker        Open        High         Low       Close  \
0     2010-01-04  GOOGL   15.689439   15.753504   15.621622   15.684434   
1     2010-01-05  GOOGL   15.695195   15.711712   15.554054   15.615365   
2     2010-01-06  GOOGL   15.662162   15.662162   15.174174   15.221722   
3     2010-01-07  GOOGL   15.250250   15.265265   14.831081   14.867367   
4     2010-01-08  GOOGL   14.814815   15.096346   14.742492   15.065566   
...          ...    ...         ...         ...         ...         ...   
3759  2024-12-10  GOOGL  182.850006  186.360001  181.050003  185.169998   
3760  2024-12-11  GOOGL  185.309998  195.610001  184.850006  195.399994   
3761  2024-12-12  GOOGL  195.000000  195.179993  191.710007  191.960007   
3762  2024-12-13  GOOGL  191.009995  192.729996  189.639999  189.820007   
3763  2024-12-16  GOOGL  192.869995  199.000000  192.619995  196.660004   

Price   Adj_Close     Volume  
0       15.627782   78169752  
1       15.558963  120067812  
2     

Price       Date Ticker        Open        High         Low       Close  \
0     2010-01-04   AAPL    7.622500    7.660714    7.585000    7.643214   
1     2010-01-05   AAPL    7.664286    7.699643    7.616071    7.656429   
2     2010-01-06   AAPL    7.656429    7.686786    7.526786    7.534643   
3     2010-01-07   AAPL    7.562500    7.571429    7.466071    7.520714   
4     2010-01-08   AAPL    7.510714    7.571429    7.466429    7.570714   
...          ...    ...         ...         ...         ...         ...   
3759  2024-12-10   AAPL  246.889999  248.210007  245.339996  247.770004   
3760  2024-12-11   AAPL  247.960007  250.800003  246.259995  246.490005   
3761  2024-12-12   AAPL  246.889999  248.740005  245.679993  247.960007   
3762  2024-12-13   AAPL  247.820007  249.289993  246.240005  248.130005   
3763  2024-12-16   AAPL  247.990005  251.380005  247.649994  251.039993   

Price   Adj_Close     Volume  
0        6.447412  493729600  
1        6.458559  601904800  
2     

In [5]:
import sqlite3
import pandas as pd

# connect to the database
# con = sqlite3.connect(r"Z:\Data\sp500_data\Scripts\sp500_market_data.db")
con = sqlite3.connect(r"C:\Users\Jonat\Documents\GitHub\lstm-stock-price-prediction\data\stock_data.db")
# pd.read_sql_query("SELECT * FROM stock_data WHERE symbol = 'PANW'", con)

df = pd.read_sql_query("SELECT * FROM stock_data", con)
con.close()
df

,Date,Ticker,Open,High,Low,Close,Adj_Close,Volume
0,2010-01-04 00:00:00,GOOGL,15.689439,15.753504,15.621622,15.684434,15.627782,78169752
1,2010-01-05 00:00:00,GOOGL,15.695195,15.711712,15.554054,15.615365,15.558963,120067812
2,2010-01-06 00:00:00,GOOGL,15.662162,15.662162,15.174174,15.221722,15.166740,158988852
3,2010-01-07 00:00:00,GOOGL,15.250250,15.265265,14.831081,14.867367,14.813666,256315428
4,2010-01-08 00:00:00,GOOGL,14.814815,15.096346,14.742492,15.065566,15.011149,188783028
...,...,...,...,...,...,...,...,...
7523,2024-12-10 00:00:00,AAPL,246.889999,248.210007,245.339996,247.770004,247.770004,36914800
7524,2024-12-11 00:00:00,AAPL,247.960007,250.800003,246.259995,246.490005,246.490005,45205800
7525,2024-12-12 00:00:00,AAPL,246.889999,248.740005,245.679993,247.960007,247.960007,32777500
7526,2024-12-13 00:00:00,AAPL,247.820007,249.289993,246.240005,248.130005,248.130005,33155300


# EOD US Bulk

$19.99/mo
100k API calls/day
1k/min

We have API limits 100 000 requests per day. Each symbol request costs 1 API call. For example, multiple tickers request with ten symbols costs 110 API calls; 
however entire exchange request costs 100 API calls.

https://eodhd.com/financial-apis/bulk-api-eod-splits-dividends?_gl=1*zl7v8c*_gcl_au*ODg0Mjc0Nzc5LjE3MzQ0ODkzMjA.*FPAU*ODg0Mjc0Nzc5LjE3MzQ0ODkzMjA.*_ga*OTQ0NDA1Mjk4LjE3MzQ0ODkzMjA.*_ga_NRYML8NKGH*MTczNDQ4OTMxOC4xLjEuMTczNDQ4OTg4Ny4wLjAuMTA0NDk2MTg5Mw..*_fplc*SFJHcGg1SGklMkI1RGhudkNpWThwQVN4QnpuVzh3ZiUyRk1PamtsV1pEZ0F5VFpjSXlNYk5wU1RyMHVFQ3hYVWolMkY0SUZYSzNwU3dBN2lBOUNvem1LYSUyRlpVR0ZzRFhHcjFuc2klMkZRcWZaU28ycnQ0dW5ZRHdpOE1TOGRKTGxHYnh1dyUzRCUzRA..

In [1]:
import requests
import pandas as pd
import numpy as np

YOUR_API_TOKEN = '64ba54ca4a54b5.70340846'
url = f'https://eodhd.com/api/eod-bulk-last-day/US?api_token={YOUR_API_TOKEN}&fmt=json'
data = requests.get(url).json()

data

[{'code': '0P0001NGF5',
  'exchange_short_name': 'US',
  'date': '2024-12-17',
  'open': 9.4792,
  'high': 9.4792,
  'low': 9.4792,
  'close': 9.4792,
  'adjusted_close': 9.4792,
  'volume': 0},
 {'code': 'A',
  'exchange_short_name': 'US',
  'date': '2024-12-17',
  'open': 137.58,
  'high': 139.34,
  'low': 136.2,
  'close': 136.46,
  'adjusted_close': 136.46,
  'volume': 2430600},
 {'code': 'AA',
  'exchange_short_name': 'US',
  'date': '2024-12-17',
  'open': 38,
  'high': 38.87,
  'low': 37.53,
  'close': 38.44,
  'adjusted_close': 38.44,
  'volume': 4420300},
 {'code': 'AAA',
  'exchange_short_name': 'US',
  'date': '2024-12-17',
  'open': 25.16,
  'high': 25.16,
  'low': 25.13,
  'close': 25.15,
  'adjusted_close': 25.15,
  'volume': 8190},
 {'code': 'AAAAX',
  'exchange_short_name': 'US',
  'date': '2024-12-17',
  'open': 12.06,
  'high': 12.06,
  'low': 12.06,
  'close': 12.06,
  'adjusted_close': 12.06,
  'volume': 0},
 {'code': 'AAACX',
  'exchange_short_name': 'US',
  'date'

In [2]:
len(data)

41445

In [4]:
result = [item for item in data if item['code'] == 'NVDA']
result

[{'code': 'NVDA',
  'exchange_short_name': 'US',
  'date': '2024-12-17',
  'open': 129.09,
  'high': 131.59,
  'low': 126.86,
  'close': 130.39,
  'adjusted_close': 130.39,
  'volume': 259410263}]

In [5]:
df = pd.DataFrame(data)
df

,code,exchange_short_name,date,open,high,low,close,adjusted_close,volume
0,0P0001NGF5,US,2024-12-17,9.4792,9.4792,9.4792,9.4792,9.4792,0
1,A,US,2024-12-17,137.5800,139.3400,136.2000,136.4600,136.4600,2430600
2,AA,US,2024-12-17,38.0000,38.8700,37.5300,38.4400,38.4400,4420300
3,AAA,US,2024-12-17,25.1600,25.1600,25.1300,25.1500,25.1500,8190
4,AAAAX,US,2024-12-17,12.0600,12.0600,12.0600,12.0600,12.0600,0
...,...,...,...,...,...,...,...,...,...
41440,ZWS,US,2024-12-17,39.2800,39.4700,38.9600,38.9800,38.9800,850568
41441,ZYME,US,2024-12-17,13.5500,13.8400,13.0800,13.3700,13.3700,494102
41442,ZYXI,US,2024-12-17,8.4500,8.6200,8.3600,8.4000,8.4000,97000
41443,ZZHGF,US,2024-12-17,2.4000,2.4000,2.4000,2.4000,2.4000,0


## All Splits Bulk EOD

In [6]:
import requests
import pandas as pd
import numpy as np

YOUR_API_TOKEN = '64ba54ca4a54b5.70340846'

url = f'https://eodhd.com/api/eod-bulk-last-day/US?api_token={YOUR_API_TOKEN}&type=splits&fmt=json'
splits = requests.get(url).json()

splits

[{'code': 'BWNAD',
  'exchange': 'US',
  'date': '2024-12-17',
  'split': '1.000000/20.000000'},
 {'code': 'CMGHF',
  'exchange': 'US',
  'date': '2024-12-17',
  'split': '1.000000/20.000000'}]

## All Dividends Bulk EOD

In [7]:
url = f'https://eodhd.com/api/eod-bulk-last-day/US?api_token={YOUR_API_TOKEN}&type=dividends&fmt=json'
dividends = requests.get(url).json()
dividends

[{'code': 'AKRYY',
  'exchange': 'US',
  'date': '2024-12-17',
  'dividend': '3.82353',
  'currency': 'USD',
  'declarationDate': '2024-10-31',
  'recordDate': '2024-11-23',
  'paymentDate': '2024-12-16',
  'period': 'Annual',
  'unadjustedValue': '3.8235300000'},
 {'code': 'APH',
  'exchange': 'US',
  'date': '2024-12-17',
  'dividend': '0.16500',
  'currency': 'USD',
  'declarationDate': '2024-10-31',
  'recordDate': '2024-12-17',
  'paymentDate': '2025-01-08',
  'period': 'Quarterly',
  'unadjustedValue': '0.1650000000'},
 {'code': 'ARES',
  'exchange': 'US',
  'date': '2024-12-17',
  'dividend': '0.93000',
  'currency': 'USD',
  'declarationDate': '2024-11-01',
  'recordDate': '2024-12-17',
  'paymentDate': '2024-12-31',
  'period': 'Quarterly',
  'unadjustedValue': '0.9300000000'},
 {'code': 'ATVDY',
  'exchange': 'US',
  'date': '2024-12-17',
  'dividend': '0.22082',
  'currency': 'USD',
  'declarationDate': None,
  'recordDate': None,
  'paymentDate': None,
  'period': None,
  '

# Historical data

Please note, that OHLC we provide in raw adjusted neither to splits nor to dividends, 
***while adjusted closes are adjusted to both splits and dividends, and volume is adjusted to splits. 

If you need OHLC adjusted only to splits then it’s better to use our Technical API with the ‘function=splitadjusted’ parameter.

MCD.US consists of two parts: {SYMBOL_NAME}.{EXCHANGE_ID}, then you can use, for example, MCD.MX for Mexican Stock Exchange. or MCD.US for NYSE. Check the list of supported exchanges to get more information about the stock markets we do support.

api_token – your own API KEY, which you will get after you subscribe to our services.
fmt – the output format. Possible values are ‘csv’ for CSV output and ‘json’ for JSON output. Default value: ‘csv’.
period – use ‘d’ for daily, ‘w’ for weekly, ‘m’ for monthly prices. By default, daily prices will be shown. Data aggregated weekly and monthly also shows weekly and monthly volume.
order – use ‘a’ for ascending dates (from old to new), ‘d’ for descending dates (from new to old). By default, dates are shown in ascending order.
from and to – the format is ‘YYYY-MM-DD’. If you need data from Jan 5, 2017, to Feb 10, 2017, you should use from=2017-01-05 and to=2017-02-10.m

In [9]:
import requests
import json

YOUR_API_TOKEN = '64ba54ca4a54b5.70340846'

url = f'https://eodhd.com/api/eod/NVDA.US?from=2010-01-01&to=2024-12-18&period=d&api_token={YOUR_API_TOKEN}&fmt=json'
nvda = requests.get(url).content

# Decode the byte string and load JSON
decoded_data = json.loads(nvda.decode('utf-8'))
nvda = pd.DataFrame(decoded_data)
nvda

b'[{"date":"2010-01-04","open":18.51,"high":18.62,"low":18.11,"close":18.49,"adjusted_close":0.4239,"volume":800352792},{"date":"2010-01-05","open":18.42,"high":18.96,"low":18.42,"close":18.76,"adjusted_close":0.4301,"volume":728697553},{"date":"2010-01-06","open":18.75,"high":18.92,"low":18.57,"close":18.88,"adjusted_close":0.4328,"volume":649166794},{"date":"2010-01-07","open":18.78,"high":18.86,"low":18.37,"close":18.51,"adjusted_close":0.4244,"volume":547790035},{"date":"2010-01-08","open":18.36,"high":18.682,"low":18.25,"close":18.55,"adjusted_close":0.4253,"volume":478175995},{"date":"2010-01-11","open":18.65,"high":18.73,"low":18.03,"close":18.29,"adjusted_close":0.4193,"volume":556655154},{"date":"2010-01-12","open":18.02,"high":18.09,"low":17.29,"close":17.67,"adjusted_close":0.4051,"volume":627488754},{"date":"2010-01-13","open":17.79,"high":17.97,"low":17.1,"close":17.91,"adjusted_close":0.4106,"volume":508945195},{"date":"2010-01-14","open":17.69,"high":17.82,"low":17.33,"c

# EODHD ADJ CLOSE = ADJ for SPLIT + DIV

In [12]:
import json

# Decode the byte string and load JSON
decoded_data = json.loads(nvda.decode('utf-8'))
nvda = pd.DataFrame(decoded_data)
nvda

,date,open,high,low,close,adjusted_close,volume
0,2010-01-04,18.51,18.620,18.11,18.49,0.4239,800352792
1,2010-01-05,18.42,18.960,18.42,18.76,0.4301,728697553
2,2010-01-06,18.75,18.920,18.57,18.88,0.4328,649166794
3,2010-01-07,18.78,18.860,18.37,18.51,0.4244,547790035
4,2010-01-08,18.36,18.682,18.25,18.55,0.4253,478175995
...,...,...,...,...,...,...,...
3760,2024-12-11,137.36,140.170,135.21,139.31,139.3100,184905203
3761,2024-12-12,137.08,138.440,135.80,137.34,137.3400,159211391
3762,2024-12-13,138.94,139.600,132.54,134.25,134.2500,231514891
3763,2024-12-16,134.18,134.400,130.42,132.00,132.0000,237951109


In [14]:
nvda.to_csv('nvda_eodhd.csv')

In [24]:
#Comparing Yfin vs EODHD data
stock_data = stock_data.copy().round(4)
stock_data

Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2010-01-04,0.4239,0.4622,0.4655,0.4527,0.4627,800204000
2010-01-05,0.4301,0.4690,0.4740,0.4605,0.4605,728648000
2010-01-06,0.4329,0.4720,0.4730,0.4642,0.4688,649168000
2010-01-07,0.4244,0.4627,0.4715,0.4593,0.4695,547792000
2010-01-08,0.4253,0.4638,0.4670,0.4563,0.4590,478168000
...,...,...,...,...,...,...
2024-12-11,139.3100,139.3100,140.1700,135.2100,137.3600,184905200
2024-12-12,137.3400,137.3400,138.4400,135.8000,137.0800,159211400
2024-12-13,134.2500,134.2500,139.6000,132.5400,138.9400,231514900


In [ ]:
import pandas as pd
import requests
from sqlalchemy import create_engine, text
import numpy as np
from datetime import date, timedelta, datetime
import pytz

# Database configuration
server =
database = 
username =
password = 
api_key = '64ba54ca4a54b5.70340846'

# Create database engine
engine = create_engine(f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server")

# Exchange operating hours in SGT (UTC+8)
exchange_hours = {
    "JK": {"open": "09:00", "close": "16:15"},  # Indonesia Stock Exchange
    "TW": {"open": "09:00", "close": "13:30"},  # Taiwan Stock Exchange
    "HK": {"open": "09:30", "close": "16:00"},  # Hong Kong Stock Exchange
    "US": {"open": "22:30", "close": "05:00"},  # NYSE/NASDAQ (Next Day in SGT)
    "KLSE": {"open": "09:00", "close": "17:00"},  # Bursa Malaysia
    "SHG": {"open": "09:30", "close": "15:00"},  # Shanghai Stock Exchange
}

def calculate_rsi(data, period=14):
    """Calculate RSI for a given DataFrame of OHLC data."""
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.ewm(alpha=1 / period, min_periods=period).mean()
    avg_loss = loss.ewm(alpha=1 / period, min_periods=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.round(2)

def sanitize_row(row):
    """Sanitize row values for database insertion."""
    row = row.copy()
    for key in row.keys():
        if isinstance(row[key], float) and (np.isnan(row[key]) or np.isinf(row[key])):
            row[key] = None
    return row

def delete_old_data(engine, cutoff_date):
    """Delete data older than the cutoff date."""
    try:
        with engine.connect() as conn:
            delete_query = text("""
                DELETE FROM HistoricalPrices
                WHERE Date < :cutoff_date;
            """)
            conn.execute(delete_query, {"cutoff_date": cutoff_date})
        print(f"Deleted data older than {cutoff_date}.")
    except Exception as e:
        print(f"Error deleting old data: {e}")

def get_last_market_date():
    """Get the last working day (non-weekend)."""
    today = datetime.today().date()
    while today.weekday() > 4:  # Saturday (5) or Sunday (6)
        today -= timedelta(days=1)
    return today

def get_latest_working_day():
    """Get the latest working day (excluding weekends)."""
    today = datetime.now(pytz.timezone("Asia/Singapore"))
    if today.weekday() == 5:  # Saturday
        return (today - timedelta(days=1)).strftime("%Y-%m-%d")
    elif today.weekday() == 6:  # Sunday
        return (today - timedelta(days=2)).strftime("%Y-%m-%d")
    return today.strftime("%Y-%m-%d")

def is_market_closed(exchange):
    """Check if the market is closed for a specific exchange."""
    now = datetime.now(pytz.timezone("Asia/Singapore"))

    open_time = datetime.strptime(exchange_hours[exchange]["open"], "%H:%M").time()
    close_time = datetime.strptime(exchange_hours[exchange]["close"], "%H:%M").time()

    # Market is closed if the current time is after close_time
    if close_time < open_time:  # Handle markets closing past midnight (e.g., US)
        return now.time() > close_time or now.time() < open_time
    else:
        return now.time() > close_time

def update_historical_prices_and_rsi(engine, tickers, start_date, end_date):
    """Fetch missing historical data and calculate RSI."""
    for ticker in tickers:
        print(f"Fetching data for {ticker} from {start_date} to {end_date}")
        url = f"https://eodhd.com/api/eod/{ticker}?from={start_date}&to={end_date}&period=d&api_token={api_key}&fmt=json"
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Failed to fetch data for {ticker}. Status: {response.status_code}")
            continue

        data = response.json()
        if not data:
            print(f"No data returned for {ticker}.")
            continue

        #print(f"Data fetched for {ticker}: {data}")
        df = pd.DataFrame(data)

        # Ensure required columns exist
        required_columns = ['date', 'open', 'high', 'low', 'close', 'adjusted_close', 'volume']
        if not all(col in df.columns for col in required_columns):
            print(f"Data for {ticker} is missing required columns. Skipping.")
            continue

        df.rename(columns={
            'date': 'Date',
            'open': 'Open',
            'high': 'High',
            'low': 'Low',
            'close': 'Close',
            'adjusted_close': 'AdjustedClose',
            'volume': 'Volume'
        }, inplace=True)

        df['Ticker'] = ticker
        df['RSI'] = calculate_rsi(df)

        # Insert data into database
        with engine.connect() as conn:
            for _, row in df.iterrows():
                row = sanitize_row(row)
                #print(f"Inserting/Updating row for {ticker}: {row}")
                insert_query = text("""
                    INSERT INTO HistoricalPrices ([Ticker], [Date], [Open], [High], [Low], [Close], [AdjustedClose], [Volume], [RSI])
VALUES (:Ticker, :Date, :Open, :High, :Low, :Close, :AdjustedClose, :Volume, :RSI);
                """)
                conn.execute(insert_query, {
                    "Ticker": row['Ticker'],
                    "Date": row['Date'],
                    "Open": row['Open'],
                    "High": row['High'],
                    "Low": row['Low'],
                    "Close": row['Close'],
                    "AdjustedClose": row['AdjustedClose'],
                    "Volume": row['Volume'],
                    "RSI": row['RSI']
                })

    print(f"Update complete for tickers: {tickers}")

def run_daily_update():
    """Run daily updates for all US tickers and maintain 3 years of data."""
    try:
        last_market_date = get_last_market_date()  # Get the last valid market date
        print(f"Last market date: {last_market_date}")

        # Fetch tickers with outdated data for US market
        outdated_query = text("""
            SELECT Ticker, MAX(Date) AS LatestDate
            FROM HistoricalPrices
            WHERE Ticker LIKE '%.US'  -- Filter for US tickers
            GROUP BY Ticker
            HAVING MAX(Date) <= :last_market_date
        """)
        with engine.connect() as conn:
            outdated_tickers = pd.read_sql(outdated_query, conn, params={"last_market_date": last_market_date})

        if outdated_tickers.empty:
            print("All tickers are up-to-date.")
            return

        print(f"Found {len(outdated_tickers)} tickers with outdated data.")

        for _, row in outdated_tickers.iterrows():
            ticker = row['Ticker']
            last_date = row['LatestDate']
            update_historical_prices_and_rsi(engine, [ticker], last_date + timedelta(days=1), last_market_date)

    except Exception as e:
        print(f"Error running daily update: {e}")

#Run Daily update
run_daily_update()

# YFIN ADJ CLOSE = ADJ for SPLIT + DIV

## YFIN OHLC ALSO ADJ for SPLIT + DIV

In [23]:
import yfinance as yf

stock_data = yf.download("NVDA", start='2010-01-01', end='2024-12-18')
stock_data.columns = stock_data.columns.droplevel(1)
stock_data

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2010-01-04,0.423923,0.462250,0.465500,0.452750,0.462750,800204000
2010-01-05,0.430113,0.469000,0.474000,0.460500,0.460500,728648000
2010-01-06,0.432864,0.472000,0.473000,0.464250,0.468750,649168000
2010-01-07,0.424381,0.462750,0.471500,0.459250,0.469500,547792000
2010-01-08,0.425298,0.463750,0.467000,0.456250,0.459000,478168000
...,...,...,...,...,...,...
2024-12-11,139.309998,139.309998,140.169998,135.210007,137.360001,184905200
2024-12-12,137.339996,137.339996,138.440002,135.800003,137.080002,159211400
2024-12-13,134.250000,134.250000,139.600006,132.539993,138.940002,231514900


# Tradingview "UNADJ" ADJ for SPLITS
# ADJ adjusts for dividends